# llama.cpp tutorial

In [1]:
 # NOT WORKING (ALL QWEN MODELS)
# model = r"C:\Users\t203771\Downloads\qwen2-0_5b-instruct-q4_k_m.gguf"

# WORKING
model = r"C:\Users\t203771\Downloads\Mistral-7B-Instruct-v0.3.Q5_K_M.gguf"
model = r"C:\Users\t203771\Downloads\bling-phi-3.gguf"

In [2]:
from llama_cpp import Llama

In [3]:
try:
    del llm
except:
    pass

llm = Llama(
    model_path = model,
    #*,

    # Model Params
    n_gpu_layers = 0,
    split_mode = 1,
    main_gpu = 0,
    tensor_split = None,
    vocab_only = False,
    use_mmap = True,
    use_mlock = False,
    kv_overrides = None,

    # Context Params
    seed = 4294967295,
    n_ctx = 0, #512,
    n_batch = 512,
    n_threads = None,
    n_threads_batch = None,
    rope_scaling_type = -1,
    pooling_type = -1,
    rope_freq_base = 0.0,
    rope_freq_scale = 0.0,
    yarn_ext_factor = -1.0,
    yarn_attn_factor = 1.0,
    yarn_beta_fast = 32.0,
    yarn_beta_slow = 1.0,
    yarn_orig_ctx = 0,
    logits_all = False,
    embedding = False,
    offload_kqv = True,
    flash_attn = False,

    # Sampling Params
    last_n_tokens_size = 64,

    # Lora Params
    lora_base = None,
    lora_scale = 1.0,
    lora_path = None,

    # BAckend Params
    numa = False,

    # Chat Format Params
    chat_format = None,
    chat_handler = None,

    # Speculative Decoding
    draft_model = None,

    # Tokenizer Override
    tokenizer = None,

    # KV cache quantization
    type_k = None,
    type_v = None,

    # # Misc
    verbose = True,

    # # Extra Params
    #**kwargs,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from C:\Users\t203771\Downloads\bling-phi-3.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count_kv u

In [4]:
dir(llm)

['_Llama__backend_initialized',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_batch',
 '_c_tensor_split',
 '_candidates',
 '_create_completion',
 '_ctx',
 '_input_ids',
 '_mirostat_mu',
 '_model',
 '_n_ctx',
 '_n_vocab',
 '_scores',
 '_token_eos',
 '_token_nl',
 'cache',
 'chat_format',
 'chat_handler',
 'context_params',
 'create_chat_completion',
 'create_chat_completion_openai_v1',
 'create_completion',
 'create_embedding',
 'ctx',
 'detokenize',
 'draft_model',
 'embed',
 'eval',
 'eval_logits',
 'eval_tokens',
 'from_pretrained',
 'generate',
 'input_ids',
 'kv_overrides',
 'last_n_tokens_size',
 'load_state',
 'l

In [5]:
prompt = "Olá!"

messages = [
            #{'role': 'system', 'content': ''},
            # {'role': 'assistant', 'content': ''},
            {'role': 'user', 'content': prompt},
            # {'role': 'assistant', 'content': 'Follows answer in Chinese:'},
           
           ]

output = llm.create_chat_completion(
    messages=messages,
    )

print(output['choices'][0]['message']['content'])


llama_print_timings:        load time =     991.96 ms
llama_print_timings:      sample time =       8.71 ms /    40 runs   (    0.22 ms per token,  4594.53 tokens per second)
llama_print_timings: prompt eval time =     991.89 ms /    11 tokens (   90.17 ms per token,    11.09 tokens per second)
llama_print_timings:        eval time =    4509.53 ms /    39 runs   (  115.63 ms per token,     8.65 tokens per second)
llama_print_timings:       total time =    5620.82 ms /    50 tokens


Olá! Como posso ajudá-lo hoje? Se tiver alguma pergunta ou necessidade, estou aqui para oferecer minha assistência.


In [10]:
llm.detokenize(llm._input_ids)

b'Ol\xc3\xa1!  Oi! Tudo bem? Como posso ajudar? Eu sou um modelo de intelig\xc3\xaancia artificial e estou aqui para responder suas perguntas ou ajudar com informa\xc3\xa7\xc3\xb5es. Vamos come\xc3\xa7ar?'

In [7]:
llm._input_ids

array([    1,     1,     3,  7780, 29588, 29576, 29473,     4,  1219,
       29478, 29576,  1088, 10065, 28879, 29572, 25196,  1745,  1435,
        1032, 14810,  1051, 29572, 28835,  6039,  3973,  2997, 29477,
        1108, 10336,  1094, 16962, 19046,  1085,  1702,  1048, 11786,
       29478,  3414,  3371,  1594,  1287,  1061,  1428, 29489,  3562,
        1061,  4234,  1032, 14810, 29588, 29501,  1499,  1200,  5995,
       29476, 11083, 29491,  1859,  5235,  1046,  1051,  1108, 24139,
       10425,  1645,  3016, 26981, 17962, 29493,  2727,  5036, 29493,
       12390,  1168, 10809,  1645, 26174,  1031,  1428, 29489,  3562,
        1061, 29576,   781,   781, 29521,  1039, 19456, 29477, 29493,
        1195,  8669, 29644,  5235,  8450, 15459,  1031,  3886, 26174,
        1031,  6680,  2848,  3924, 29493,  4299, 25620, 12736,  1050,
       11095,  1229,  3562,  1153,  3414, 17868,  1051,  1032,  9738,
        2520,  1444, 29474,  5575, 29491,  1859,  5235,  1046,  1051,
        1108, 24139,

In [8]:
len(llm._input_ids)

148

In [9]:
llm.n_tokens

148